In [1]:
import numpy as np
import icechunk
from nimblend import Array, to_zarr, from_zarr, to_icechunk, from_series, to_series

In [2]:
# Create two arrays with different coordinates
data1 = np.array([[1, 2], [3, 4]])
coords1 = {'x': ['a', 'b'], 'y': [0, 1]}
arr1 = Array(data1, coords1)

data2 = np.array([[10, 20, 30], [40, 50, 60]])
coords2 = {'x': ['b', 'c'], 'y': [0, 1, 2]}
arr2 = Array(data2, coords2)

# Operation automatically aligns coordinates
result = arr1 + arr2
print(result)

# Create a lazy array with Dask
lazy_arr = Array(data1, coords1, chunks="auto")
print(lazy_arr.is_lazy)  # True

# Save array to Zarr format
to_zarr(result, "result.zarr")

# Load from Zarr
loaded = from_zarr("result.zarr")

Array((3, 3), {'x': (3,), 'y': (3,)})
True


In [3]:
# Create some sample data
data1 = np.array([
    [[1, 2, 3], [4, 5, 6]],
    [[7, 8, 9], [10, 11, 12]],
])

data2 = np.array([
    [[13, 14, 15], [16, 17, 18]],
    [[19, 20, 21], [22, 23, 24]],
])

# Define coordinates with strings for dimensions
coords1 = {
    'x': np.array(['A', 'B']),
    'y': np.array(['one', 'two']),
    'z': np.array(['i', 'ii', 'iii'])
}

coords2 = {
    'x': np.array(['C', 'D']),
    'y': np.array(['three', 'four']),
    'z': np.array(['iv', 'v', 'vi'])
}

# Create NimbleNd Arrays
arr3 = Array(data1, coords=coords1, dims=['x', 'y', 'z'], name='Array 1')
arr4 = Array(data2, coords=coords2, dims=['x', 'y', 'z'], name='Array 2')

In [4]:
storage = icechunk.local_filesystem_storage("/workspaces/nimblend/dataset")
repo = icechunk.Repository.open_or_create(storage)
session = repo.writable_session("main")

In [5]:
to_icechunk(result, session)

In [6]:

# Test addition
print("Adding arrays:")
result_add = arr3 + arr4
print(result_add)

# Export to Zarr format
print("\nExporting Array 3 to Zarr format...")
to_zarr(arr3, 'arr3.zarr')

# Import from Zarr format
print("\nImporting Array from Zarr format...")
imported_arr3 = from_zarr('arr3.zarr')
print(imported_arr3)

# Example usage of to_icechunk would go here (ensure icechunk is installed)
# Assuming a session is created and available as `session`
# print("\nExporting Array 1 to IceChunk format...")
# to_icechunk(arr1, session, group='example_group')

# Example series conversion (using strings for Series coordinates)
print("\nConverting Array 1 to a pandas Series:")
series_arr3 = to_series(arr3, format='pandas')
print(series_arr3)

print("\nConverting Series back to NimbleNd Array:")
converted_arr3 = from_series(series_arr3)
print(converted_arr3)

Adding arrays:
Array((4, 4, 6), {'y': (4,), 'x': (4,), 'z': (6,)}, name='Array 1')

Exporting Array 3 to Zarr format...

Importing Array from Zarr format...
Array((2, 2, 3), {'x': (2,), 'y': (2,), 'z': (3,)}, name='Array 1')

Converting Array 1 to a pandas Series:
x  y    z  
A  one  i       1
        ii      2
        iii     3
   two  i       4
        ii      5
        iii     6
B  one  i       7
        ii      8
        iii     9
   two  i      10
        ii     11
        iii    12
Name: Array 1, dtype: int64

Converting Series back to NimbleNd Array:
Array((2, 2, 3), {'x': (2,), 'y': (2,), 'z': (3,)}, name='value')


In [7]:
converted_arr3

Dimensions,Coordinates
x,['A' 'B']
y,['one' 'two']
z,['i' 'ii' 'iii']


In [8]:
imported_arr3

Dimensions,Coordinates
x,['A' 'B']
y,['one' 'two']
z,['i' 'ii' 'iii']


In [9]:
lazy_arr

Dimensions,Coordinates
x,['a' 'b']
y,[0 1]


In [10]:
lazy_arr.compute()

Dimensions,Coordinates
x,['a' 'b']
y,[0 1]


In [11]:
lazy_arr.persist()

Dimensions,Coordinates
x,['a' 'b']
y,[0 1]


In [12]:
lazy_arr

Dimensions,Coordinates
x,['a' 'b']
y,[0 1]
